In [0]:
# Notebook: /Shared/bronze/customer_account_ingest
# Arguments: raw_path (abfss path to CSV), bronze_path (abfss path to output Delta)
# Example raw_path: abfss://staging@{account}.dfs.core.windows.net/raw/customer_account/customerAccount.csv

from pyspark.sql.functions import current_timestamp, lit

# dbutils widgets (if invoked from ADF) 

dbutils.widgets.text("bronze_path", "")
dbutils.widgets.text("usecasefolder", "")




In [0]:
bronze_path = dbutils.widgets.get("bronze_path") 
usecasefolder = dbutils.widgets.get("usecasefolder")


print(f"bronze_path:-{bronze_path}")
print(f"usecasefolder:-{usecasefolder}")

# Read raw Parquet (pipe separator is not applicable to Parquet)
df_bronze = spark.read.parquet(bronze_path)

# Add ingestion metadata columns
df_silver = df_raw.withColumn("_ingest_at", current_timestamp()) \
                  .withColumn("_source_file", lit(bronze_path))

# Basic sanity check: show schema & sample
print("Schema (raw):")
df_silver.printSchema()



base_path = bronze_path.split('/', 3)[0:3]
result = "/".join(base_path) + "/" + "silver"+ "/" + usecasefolder

print(f"splitpath:-{result}")


# Write to Bronze Delta (append mode)
(df_silver.write
  .format("delta")
  .mode("append")
  .option("mergeSchema","false")
  .save(result))

print(f"Wrote Bronze Delta to: {result}")



#dbutils.notebook.exit(f"success:- Wrote Bronze Delta to: {result}")